In [1]:
from keras.preprocessing import sequence
from keras.datasets import imdb
from keras import layers, models

Using TensorFlow backend.


In [2]:
class Data:
    def __init__(self, max_features=20000, maxlen=80):
        (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

        # 입력 문장의 최대 길이(80)보다 클 경우 잘라내고, 작을 경우 앞에 padding(0)을 추가
        x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
        x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test


# data lookup
data = Data()
print('train:', len(data.y_train), ', test:', len(data.y_test))
print('label:', data.y_train[20000:20005])
print('features:', data.x_train[20000:20005])

17465344/17464789 [==============================] - 25s 1us/step
train: 25000 , test: 25000
label: [0 0 0 0 1]
features: [[   10    10    17     6    22  1985   255    11   631    62    28    77
    128    48     2   109    66    16     6   668  1985    19    41   861
      2     6    87   227     7  3588    21    17   230    17  3469  2391
      5   541  1554   140    14    31     9   254     8  1559 16128     2
      9   345  2516    42     2  5758   469     4    22    63   944     6
   1257  1166     7  2172   599  6203     2     2    17     2     5  8548
   2234  5252    17     2    26    52   696  1850]
 [  820   910  1030     8  5817   183    56    11  2716     7    68 12478
   4697    10    10   910    70  4146     4   118   927     4   118  1180
      5   907    21   131    36  5817   183    56    14    20     9   595
      4   619   155     9    15    13  1781   910    11    68  7671   127
     24    60   124    54     6    20     9    52    42    78    10    10
     12     9

#### 단어가 2000개면 숫자가 numeric X data의 range가 0~2000 이 되는 encoding 개념.